In [ ]:
import sys
import transformers
import pandas as pd
import numpy as np
import glob
import nltk
import torch
import math
import random
import re
import argparse
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
import os
doc_dir = '/content/drive/MyDrive/abs/train/judgement'
summary_dir = '/content/drive/MyDrive/abs/train/summary'

def load_documents_and_summaries(doc_dir, summary_dir):
    documents = []
    summaries = []


    document_files = set(f for f in os.listdir(doc_dir) if f.endswith('.txt'))
    summary_files = set(f for f in os.listdir(summary_dir) if f.endswith('.txt'))


    for file in document_files:
        if file in summary_files:

            with open(os.path.join(doc_dir, file), 'r', encoding='utf-8') as doc_file:
                document_content = doc_file.read()
            with open(os.path.join(summary_dir, file), 'r', encoding='utf-8') as summary_file:
                summary_content = summary_file.read()


            documents.append(document_content)
            summaries.append(summary_content)

    return documents, summaries
documents, summaries = load_documents_and_summaries(doc_dir, summary_dir)
'''

KeyboardInterrupt: 

In [ ]:
'''import csv

data = list(zip(documents, summaries))

filename = 'dataset.csv'

# Write to the CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Header1', 'Header2'])  # Writing headers if needed
    writer.writerows(data)

print(f"Data written to {filename}")
'''

In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv('/content/drive/My Drive/dataset.csv')

In [ ]:
df.head(1)

,Documents,Summaries
0,minal Appeals No. 76 82 of 1965.\nAppeals from...,Section 18 of the Suppression of Immoral Traff...


In [ ]:
pip install  datasets transformers

In [ ]:
!pip install pyarrow==5.0.0


  Using cached pyarrow-5.0.0.tar.gz (739 kB)
  Installing build dependencies ... canceledERROR: Operation cancelled by user


In [ ]:
!pip install --upgrade --force-reinstall datasets


  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.17-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.10.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.6 kB)
  Using cached huggingface_hub-0.25.2-py

In [ ]:
!pip install --upgrade pyarrow
import datasets
from datasets import load_dataset
dataset = load_dataset('csv', data_files='/content/drive/My Drive/dataset.csv')

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Documents', 'Summaries'],
        num_rows: 2463
    })
})

In [ ]:
split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)


train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")


Training samples: 1970
Testing samples: 493


In [ ]:
test_dataset[0]

{'Documents': 'iminal Appeal No. 30 of 1961.\nAppeal from the judgment and order dated September 6, 1960 of the Calcutta High Court in Cr.\nRevision No. 647 of 1960.\nB. Sen, P. K. Chatterjee and P. K. Bose, for the appellant.\nD. C. Roy and P. K. Mukherjee, for the respondent.\nApril 12.\nThe Judgment of the Court was delivered by KAPUR, J.\nThis is an appeal against the judgment and order of the High Court of Calcutta quashing the investigation started against the respondent in regard to offences under section 420, Indian Penal Code, and section 120B read with section 420 of the Indian Penal Code.\nOn March 26, 1960, Sub Inspector.\nB. L. Gbose of Police Inforcement Branch filed a written report before the Officer in charge Chakdha P. section, alleging that the respondent in conspiracy with three others 54 had cheated the Government of West Bengal of a sum of Rs. 20,000.\nThe respondent at the time was an Assistant cum Executive Engineer, Kancbrapara Development Area, Kalyani Divisio

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['Documents', 'Summaries'],
        num_rows: 1970
    })
    test: Dataset({
        features: ['Documents', 'Summaries'],
        num_rows: 493
    })
})

In [ ]:
pip install transformers datasets


In [ ]:
from transformers import BertTokenizer



In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, BertModel, BartForConditionalGeneration

bert_tokenizer = AutoTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
bert_model = BertModel.from_pretrained('nlpaueb/legal-bert-base-uncased')

bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset

class SummaryDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]), #list to tensor
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.encodings['labels'][idx])
        }

    def __len__(self):
        return len(self.encodings['input_ids'])


In [ ]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

def tokenize_function(examples):
    examples['Summaries'] = [str(summary) for summary in examples['Summaries']]

    inputs = tokenizer(
        examples["Documents"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    targets = tokenizer(
        examples["Summaries"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )


    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']
    }

tokenized_train_dataset = split_dataset["train"].map(tokenize_function, batched=True)
tokenized_test_dataset = split_dataset["test"].map(tokenize_function, batched=True)



In [ ]:
train_dataset = SummaryDataset(tokenized_train_dataset)
test_dataset = SummaryDataset(tokenized_test_dataset)


from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
'''# Get the first batch from the train_dataloader
for batch in train_dataloader:
    first_batch = batch
    break

# Now you can access the data in the first_batch dictionary
print(first_batch['input_ids'])  # Example: Accessing input_ids
print(first_batch['attention_mask'])  # Example: Accessing attention_mask
print(first_batch['labels'])  # Example: Accessing labels'''

"# Get the first batch from the train_dataloader\nfor batch in train_dataloader:\n    first_batch = batch\n    break\n\n# Now you can access the data in the first_batch dictionary\nprint(first_batch['input_ids'])  # Example: Accessing input_ids\nprint(first_batch['attention_mask'])  # Example: Accessing attention_mask\nprint(first_batch['labels'])  # Example: Accessing labels"

In [ ]:
from transformers import BartForConditionalGeneration, AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large").to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)


        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")

print("Training complete!")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Average Loss: 2.640979542423356
Training complete!


In [ ]:
def generate_summary(text):
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs['input_ids'], max_length=4000, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text = tokenized_train_dataset[0]['Documents']
print(generate_summary(text))


The appellant was the sole proprietor of a business known as "section N. Dutt & Co. " and issued two notices under section 80 of the Code of Civil Procedure for the supply of mangoes at Jiaganj Railway Station, every day from May 24, 1944, for ten days at the rate of 1,000 baskets per day.
On May 30, 1944 the military authorities informed the contractor that the contract had been cancelled on account of the unsatisfactory nature of the supplies.
Subsequently the appellant filed a claim for damages for over Rs. 84,000 on the ground that he had sustained a heavy loss due to the misconduct, gross negligence and carelessness on the part of the Bengal and Assam Railway administration.
The suit was resisted by the Governor General in Council, but the Union of India (respondent) contended that the two notices were not valid and sufficient, and that the appellant was not entitled to maintain the suit as the notices under


In [ ]:
model.save_pretrained('/content/drive/MyDrive')
tokenizer.save_pretrained('/content/drive/MyDrive')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/tokenizer_config.json',
 '/content/drive/MyDrive/special_tokens_map.json',
 '/content/drive/MyDrive/vocab.json',
 '/content/drive/MyDrive/merges.txt',
 '/content/drive/MyDrive/added_tokens.json')

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

save_directory = '/content/drive/MyDrive'


tokenizer = BartTokenizer.from_pretrained(save_directory)


model = BartForConditionalGeneration.from_pretrained(save_directory)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
model.eval()

generated_summaries = []
actual_summaries = []

for batch in test_dataloader:

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    with torch.no_grad():
        summary_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, num_beams=4, early_stopping=True)

    generated = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    generated_summaries.extend(generated)

    actual = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
    actual_summaries.extend(actual)

In [ ]:
!pip install evaluate
import evaluate

rouge = evaluate.load('rouge')

In [ ]:
from evaluate import load
rouge = load('rouge')
from rouge_score import rouge_scorer

In [ ]:
!pip install nltk evaluate
!pip install --force-reinstall numpy==1.25.2
!pip install evaluate --upgrade
import nltk
!pip install rouge_score
from nltk.translate import bleu_score
import evaluate

nltk.download('punkt')

  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
cudf-cu12 24.6.1 requires pyarrow<16.2.0a0,>=16.1.0, but you have pyarrow 17.0.0 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

def calculate_bleu(original, generated):
    scores = []
    for orig, gen in zip(actual_summaries, generated_summaries):

        orig_tokens = nltk.word_tokenize(orig)
        gen_tokens = nltk.word_tokenize(gen)


        score = bleu_score.sentence_bleu([orig_tokens], gen_tokens)
        scores.append(score)
    return scores


rouge = evaluate.load('rouge')

def calculate_rouge(original, generated):
    scores = rouge.compute(predictions=generated, references=original)
    return scores

bleu_scores = calculate_bleu(actual_summaries, generated_summaries)
rouge_scores = calculate_rouge(actual_summaries, generated_summaries)


print("BLEU Scores:", bleu_scores)
print("ROUGE Scores:", rouge_scores)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU Scores: [0.1932885872761737, 0.15806679374127577, 0.2178327857558575, 1.6092139539174812e-78, 0.5431453932735718, 0.0940713539742651, 0.2577307440085172, 0.06515188738820063, 0.2516815734403964, 1.6799713249147274e-78, 0.3000623861750799, 0.16438253419899262, 0.27551401625837396, 0.1151135819433661, 0.08762216398666815, 0.05948639071686123, 4.434755931828925e-155, 0.06057145345713495, 0.5580660686244475, 0.30042319114651783, 2.1504271967054382e-78, 0.09681210625627841, 1.255800910665416e-78, 0.19514631432816512, 0.1510618535913012, 1.752911800224005e-78, 0.07926116415700454, 0.4903294524282231, 0.3037783320572248, 0.44235052712296585, 0.3079512600371864, 0.1516088243570965, 1.3458872190999414e-78, 0.10802011816912092, 0.053802309398893085, 0.1772582120465654, 0.06494951888375101, 0.23031046916638137, 5.723654496146223e-155, 0.15865815419993443, 0.053955663597621, 2.0363030544801795e-78, 0.18434353850016247, 0.2913280138827994, 0.11825001169349016, 0.04625900984155647, 0.4043720681

In [ ]:
'''BLEU Scores: [0.1932885872761737, 0.15806679374127577, 0.2178327857558575, 1.6092139539174812e-78, 0.5431453932735718, 0.0940713539742651,
              0.2577307440085172, 0.06515188738820063, 0.2516815734403964, 1.6799713249147274e-78, 0.3000623861750799, 0.16438253419899262,
              0.27551401625837396, 0.1151135819433661, 0.08762216398666815, 0.05948639071686123, 4.434755931828925e-155, 0.06057145345713495,
              0.5580660686244475, 0.30042319114651783, 2.1504271967054382e-78, 0.09681210625627841, 1.255800910665416e-78, 0.19514631432816512,
              0.1510618535913012, 1.752911800224005e-78, 0.07926116415700454, 0.4903294524282231, 0.3037783320572248, 0.44235052712296585,
              0.3079512600371864, 0.1516088243570965, 1.3458872190999414e-78, 0.10802011816912092, 0.053802309398893085, 0.1772582120465654,
              0.06494951888375101, 0.23031046916638137, 5.723654496146223e-155, 0.15865815419993443, 0.053955663597621, 2.0363030544801795e-78,
              0.18434353850016247, 0.2913280138827994, 0.11825001169349016, 0.04625900984155647, 0.40437206812271975, 0.11709921338845747,
              4.533086463968084e-155, 0.1613950084180782, 1.4446150405344093e-78, 0.16837501502952912, 0.1352399289933244,
              3.2208364377701786e-155, 0.17139234341381263, 0.3370759373213762, 4.831880060174842e-155, 0.30418608000767383,
              0.0717514236756446, 0.09098681952312344, 0.13910376573800953, 0.1034180801972501, 0.14977080196475978, 1.5632720244275045e-78,
              0.2347934704946878, 0.2773044870025194, 0.2149413666493006, 0.21772163567428915, 0.08283233422563915, 0.09782141596543549,
              2.3667453417675176e-78, 0.35686616041557395, 0.17853462595575406, 0.46080005718177475, 5.772673208665528e-155, 0.19741996233532902,
              1.632936498736062e-78, 0.08022963484722281, 0.1827669041522176, 5.218381790024649e-155, 0.09825505159578724, 1.0315981400120194e-231,
              0.3374430542743561, 0.21284376833500962, 4.922088002769674e-155, 0.13480497688153606, 0.04788228118477925, 0.2525066072841661,
              0.10217735011741548, 1.13631817938959e-78, 0.2913749711415567, 1.3600901248760453e-78, 2.9991295067735775e-78, 0.4747826577588376,
              0.09342259116609009, 0.22398488468888347, 0.06203700079640945, 0.16987974648063328, 2.0979406822475038e-78, 0.060802042648735184,
              0.20304044356283835, 0.3895625481573781, 0.07278230723905438, 0.3418094272586104, 0.05004968521265101, 4.2873470459658165e-155,
              0.08759571113730337, 0.28834178391439885, 0.254082443685274, 0.32629449944539335, 0.04699492303755103, 0.1503798509475338,
              0.12323475445153678, 0.06414661050401747, 0.18278944364942573, 0.19992895203141936, 0.1035923003188843, 0.06369810666703662,
              0.3083035444496935, 0.08093465092035856, 0.08834439005876074, 0.06841608121937082, 1.3739107127321597e-78, 4.879521170109991e-155,
              0.4442813095496761, 0.3942671713982985, 0.2796414951510255, 0.1598193639941141, 0.09031266143565046, 0.17467849949201703,
              0.2858451575152312, 0.25757791487472065, 0, 0.16965162876371312, 0.044558416581482525, 0.24612908133059228, 6.224633069122851e-155
              , 0.3463954034794967, 1.698533759771355e-78, 0.24185568829640583, 0.10803939378866051, 0.09255732555437171, 0.0856874790890832,
              0.15094118722782046, 0.469393086159674, 0.179898716603355, 0.12797484901820083, 0.4314545124723786, 6.6082516737175035e-155,
              0.25617914835718675, 0.06818334194266762, 0.13514217538558632, 0.04927316123437251, 0.30374454720502037, 0.2070648141558418,
              0.0414200494458399, 1.908404691994679e-78, 1.4269924697242776e-78, 0.21984613065028252, 0.050649013250316094, 4.151132611750382e-155,
              0.27959476389470356, 0.28613786444194467, 0.10028593298099764, 0.16944196620612173, 0.598800678247296, 5.1107048151966984e-155,
              3.0827844619818468e-78, 0.04395001227384646, 0.1259454628390947, 0.05353650760408128, 0.3150160408269359, 0.2002630190090658,
              0.41455378425175643, 0.10351743384633724, 0.11821728482264325, 0.13675848266787588, 0.08851732081680777, 0.11917322848353729,
              4.7375042889412906e-155, 0.07673411165782971, 0.16118732182393355, 0.088147110823494, 1.7347258616232127e-78, 0.2919656241444065,
              0.2101983443576319, 1.4302207744633639e-78, 4.212303165997554e-155, 0.14381092027832323, 0.10662284658242678, 0.3566477868605444, 0.15803339247244078, 3.981897812932139e-155, 0.17229578039256851, 0.12151003175175461, 0.2960081148863753, 0.09185777529863207, 0.11844397731252482, 0.1537497139715524, 0.19551459949257788, 5.357895237797952e-155, 0.10101251464744151, 5.4771584153812925e-155, 0.36442489681366624, 4.265702461025784e-155, 0.19745745344778806, 0.06484712703657584, 0.38740949403190567, 1.2939430828380278e-78, 0.3769326823135244, 0.13996760604625888, 0.17259621009551626, 0.20771918121180982, 0.14040321900707187, 4.6644316252602715e-155, 0.05738744826271924, 9.807557348940825e-79, 0.06302530174928503, 0.2550303651676375, 0.04340872850222474, 1.2677332724912832e-78, 0.33579212955160037, 4.292081849136754e-155, 0.17809626199062403, 1.801764984376408e-78, 0.19158265419876033, 0.1092371999065142, 0.14706141008374948, 2.6133855065210454e-78, 0.055968675592728404, 0.08353727198189834, 0.07950894110280342, 0.11386016716127477, 0.12239844150794589, 0.08134468918220059, 0.1937046245953874, 0.20935627742414004, 2.7169901998349077e-78, 0.3410242706534153, 0.23016305393320144, 0.24223794595223827, 0.08198915278088748, 0.5551565854709237, 2.0568094786720224e-78, 0.3580609478998635, 0.29552474725646494, 5.260776998893044e-155, 0.1723756394083971, 0.3735264360080446, 0.10119977355943026, 0.06065967385312479, 4.707142878213473e-155, 1.9632123008821593e-78, 0.3177169536457236, 0.38154867502199896, 0.10387303496429823, 0.5222784181049829, 1.892518848083024e-78, 0.17394607875276794, 0.06358833451001843, 0.5406534445285707, 1.3345568719245925e-78, 0.04521856298373329, 0.2531252663502322, 0.34840713186821437, 0.19322520746775856, 0.08968067903540826, 0.10168630107390682, 0.15144151730481617, 0.2552351975310662, 0.2791841818628717, 0.07439331904857548, 0.1965171522215773, 0.16926122992561418, 0.13833758542667313, 0.1742370637108001, 0.04332730075394869, 0.2854128509193287, 0.111804358873304, 5.734939125263529e-155, 0.19106127642407883, 0.23907895984994296, 5.478310479235809e-155, 0.30885648689527007, 0.16201345477704185, 0.05015395563558039, 0.10436111865090873, 0.14650195877569483, 0.6155644059090049, 4.093656693195303e-155, 0.04700606979275042, 0.09573650382669316, 0.4038219340923106, 0.23343858578113233, 0.09315766322990698, 1.7269198111947483e-78, 0.07594724600530921, 4.807998311965606e-155, 0.27054729171726644, 5.253441210115029e-155, 0.13274033061570156, 0.08785570875601481, 0.20828244718405298, 0.07722193062582534, 0.28301617147576635, 0.15524262293607355, 2.1535568780180754e-78, 0.2620192822306019, 0.1744329782298451, 0.1711959295105451, 0.1415344584111205, 0.06237657308696711, 0.13005039725356196, 3.4539290323188128e-155, 4.212421023080109e-155, 1.5094114763651908e-78, 0.04877418650798555, 0.18994394889036317, 0.36038768111844355, 0.4704332894417627, 0.14132109200379508, 0.08360058736910576, 0.0448189298907735, 0.054583020300053825, 4.994803113241671e-155, 0.1721512395880948, 0.1862672359582918, 0.28969470987813845, 1.1455056843471962e-78, 0.08215909739223486, 0.09273334885767033, 4.7246586929036255e-155, 4.584895596309517e-155, 0.1639064593928471, 0.0953337329962604, 0.19896512824811474, 0.042092637167762154, 0.1500818541114657, 0.17931710599226466, 0.07277799904348728, 4.368048869756373e-155, 0.06359269312399486, 0.39529091498948077, 0.2156856823064808, 4.728796941442346e-155, 0.03490886168574304, 0.061825740544944725, 0.16256319619543166, 0.16299711343436138, 1.6697269724880712e-78, 0.19765496629564291, 0.13530753862084482, 0.2006327927646805, 0.3454361488295174, 0.21386384140664694, 0.2819551085900857, 0.05215095332243106, 1.2204243794056445e-78, 0.07378113701655048, 0.07686003108254455, 0.2245995266206816, 0.22675399911602903, 0.171386677360159, 0.38578831087633475, 5.139505843077844e-155, 0.48542432355736215, 1.866151143365825e-78, 0.1235467047322923, 0.304753099030368, 5.876935367307856e-155, 0.1386634402197918, 1.6605102316678907e-78, 2.3995801109667714e-78, 0.09524462260195535, 2.1630558358515285e-78, 0.09246561079757709, 0.18799650504736948, 1.3691765585696824e-78, 0.12345025750852548, 0.13692636062808303, 0.3721409883565713, 0.11104479005246592, 0.23158856221315452, 0.2544574741430079, 0.1814041030366095, 0.21094369286512818, 0.39403424830245737, 0.10231622504655295, 0.24078030976898618, 0.10676737218672136, 6.088652923783793e-155, 0.07234539969917397, 0.17267412648186456, 0.19845263867714022, 0.0887612794833903, 0.35969659414966776, 0.049739881934800376, 0.2372350162128614, 0.32772843566646276, 0.09028010282074002, 5.088788558612117e-155, 0.08723869062122971, 0.3930171986554486, 0.14455683162952612, 0.10664238352261128, 0.11098462883034825, 5.752115154501478e-155, 0.07641719273709253, 0.2699032826052456, 0.11888422643873697, 1.7550065774933343e-78, 0.20061831474173866, 0.07257355411383636, 4.793315061671714e-155, 0.10401718851352745, 4.366616205694601e-155, 0.28788637206141987, 0.12585698889683172, 0.1538106929522615, 0.21458573358127073, 0.06342879870764748, 0.19197237790795155, 0.18836213660236184, 0.2378625911460677, 0.1666590362824568, 0.2693963044555114, 0.2673669881972678, 0.36780203576605, 1.5606937776698067e-78, 1.7462909665862644e-78, 5.6937104824858e-155, 0.16263045601964365, 0.1314180973841296, 0.23853878611713578, 1.52018576094114e-78, 0.18368229400543856, 0.0906759735034126, 0.35806949458378756, 5.199619974999846e-155, 0.3072624812522378, 0.17572883894082977, 4.921765457514497e-155, 0.11152830158430571, 5.425230030075562e-155, 0.14587976182154636, 0.11206264910809553, 0.047170608056152606, 0.31923876926357725, 1.5247095613463504e-78, 0.14046579268051027, 0.1525823315243838, 0.11997649405139396, 1.2989580901019031e-78, 0.273971867025172, 0.1721957219668327, 4.943593139278472e-155, 4.7753034487398274e-155, 0.2350831220284903, 0.28863331474128007, 0.177659646834783, 0.36549632243104174, 0.28516792848967654, 0.052268689276109837, 3.734286338932729e-155, 0.08345217501675024, 0.0595152910980671, 0.08772485469558462, 0.28087871796334185, 0.37188199355167434, 0.2038558585927781, 0.0665881459819505, 0.12432464976741713, 0.06069438420924982, 0.08470406003656407, 0.10040355451454681, 0.14661576432401083, 0.15253920764035475, 0.10144651236008818, 4.439317159086504e-155, 1.1710551903353398e-78, 2.3884310017602493e-78, 0.1026263205340462, 0.062255452685129324, 0.09405206929864028, 1.5222331350524324e-78, 0.17815284540109544, 0.23047905461603085, 0.21583991748731432, 0.16502346876469734, 0.13511534713966275, 0.2459534916304371, 0.155568056722597, 0.08968874703438949]'''


'BLEU Scores: [0.1932885872761737, 0.15806679374127577, 0.2178327857558575, 1.6092139539174812e-78, 0.5431453932735718, 0.0940713539742651,\n              0.2577307440085172, 0.06515188738820063, 0.2516815734403964, 1.6799713249147274e-78, 0.3000623861750799, 0.16438253419899262,\n              0.27551401625837396, 0.1151135819433661, 0.08762216398666815, 0.05948639071686123, 4.434755931828925e-155, 0.06057145345713495,\n              0.5580660686244475, 0.30042319114651783, 2.1504271967054382e-78, 0.09681210625627841, 1.255800910665416e-78, 0.19514631432816512,\n              0.1510618535913012, 1.752911800224005e-78, 0.07926116415700454, 0.4903294524282231, 0.3037783320572248, 0.44235052712296585,\n              0.3079512600371864, 0.1516088243570965, 1.3458872190999414e-78, 0.10802011816912092, 0.053802309398893085, 0.1772582120465654,\n              0.06494951888375101, 0.23031046916638137, 5.723654496146223e-155, 0.15865815419993443, 0.053955663597621, 2.0363030544801795e-78,\n   

In [ ]:
actual_summaries[0]

'A Sub Inspcctor of Police, Enforcement Branch, filed a report before the Police Officer in charge of a Police Station alleging that the respondent along with three others committed offences under ss.420, 120B read with s.420 Indian Penal Code.\nThereupon a First Information Report was drawn up and investigation was started.\nThe respondent surrendered before the judicial Magistrate and he was released on bail.\nSubsequently he filed an application in the High Court under sections 439 and 561 A of the Criminal Procedure Code to get the case pending before the judicial Magistrate arising out of the 53 case registered in the Police Station'

In [ ]:
generated_summaries[0]

'The respondent was an Assistant cum Executive Engineer, Kancbrapara Development Area, Kalyani Division.\nOn March 26, 1960, Sub Inspector. Gbose of Police Inforcement Branch filed a written report before the Officer in charge of the Chakdah Police Station alleging that the respondent in conspiracy with three others had cheated the Government of West Bengal of a sum of Rs. 20,000.\nThe police started investigation and the respondent surrendered in the court of the Judicial Magistrate at Ranaghat and was released on bail.\nHe filed a petition under sections 439 and 561A of the Criminal Procedure Code and prayed for a rule against the District Magistrate, Nadia, to show cause why the judicial'